# Lesson 5.1: Introduction to Memory

---

In previous lessons, we built Large Language Model (LLM) applications that could answer questions or perform tasks. However, if you tried to have a multi-turn conversation with them, you would notice a major limitation: they don't "remember" previous interactions. Each query is processed as a completely new conversation. This is the **statelessness problem** of LLMs.

## 1. The Statelessness Problem in LLM Conversations

Large Language Models (LLMs) are inherently **stateless**. This means that every time you send a prompt to an LLM, it processes that prompt independently, without any memory of previous prompts or responses within the same "conversation."

* **Example:**
    * **You:** "What is the capital of France?"
    * **LLM:** "The capital of France is Paris."
    * **You:** "What is its population?"
    * **LLM (without Memory):** "I don't know what 'it' refers to." (Because it has forgotten Paris).

For an LLM to maintain a fluid and contextual conversation, we need a mechanism to provide the conversation history back in each LLM call. This mechanism is called **Memory**.




---

## 2. Why Memory is Needed in Chatbot and Conversational Applications

**Memory** is an essential component for any LLM application that requires maintaining context across multiple turns, such as chatbots, virtual assistants, or multi-step interactive applications.

* **Context Preservation:** Allows the LLM to understand follow-up questions that depend on previously discussed information.
* **Better User Experience:** When the LLM "remembers," the conversation becomes more natural, fluid, and effective, similar to chatting with a human.
* **Complex Problem Solving:** For multi-step tasks or those requiring state tracking (e.g., placing an order, filling out a form), Memory is indispensable.
* **Personalization:** Enables the LLM to provide more tailored responses based on the history of interactions with a specific user.


---

## 3. Types of Memory in LangChain

LangChain offers various types of Memory, each with its own way of storing and retrieving conversation history, suitable for different needs.

### 3.1. `ChatMessageHistory` (Most Basic)

* **Concept:** A foundational class for storing a list of `Message` objects (e.g., `HumanMessage`, `AIMessage`). It's simply a buffer, with no complex logic.
* **Usage:** You add messages to it and retrieve the entire list when needed.

In [ ]:
from langchain_core.chat_history import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

history = ChatMessageHistory()

history.add_user_message("Hello, how are you?")
history.add_ai_message("I'm fine, thank you! Do you have any questions?")
history.add_user_message("I want to ask about LangChain.")

print(history.messages)

### 3.2. `ConversationBufferMemory` (Stores Full History)

* **Concept:** Stores the entire conversation history (all messages) in a buffer.
* **Pros:** Simple, easy to use, retains full context.
* **Cons:** Can become very large and exceed LLM token limits if the conversation is too long.
* **Usage:** Integrates directly into Chains or Agents.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Initialize ConversationBufferMemory
memory = ConversationBufferMemory()

# Initialize ConversationChain with Memory
# verbose=True to see the chain's operation
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

print("--- ConversationBufferMemory Example ---")

# Conversation turn 1
response_1 = conversation.invoke({"input": "What is the capital of France?"})
print(f"You: What is the capital of France?")
print(f"AI: {response_1['response']}")

# Conversation turn 2 (LLM remembers context)
response_2 = conversation.invoke({"input": "What is its population?"})
print(f"You: What is its population?")
print(f"AI: {response_2['response']}")

print("\n--- History in Memory ---")
print(memory.buffer) # View the entire stored history

### 3.3. `ConversationBufferWindowMemory` (Stores Last N Messages)

* **Concept:** Stores only a window of the last `k` messages. When a new message arrives, the oldest message is discarded.
* **Pros:** Prevents conversation history from becoming too long, helping control token limits.
* **Cons:** Can lose important context if that information falls outside the `k`-message window.
* **When to Use:** When you need to maintain short-term context and want to avoid exceeding token limits.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# Initialize ConversationBufferWindowMemory with k=1 (remembers only the last 1 message pair)
window_memory = ConversationBufferWindowMemory(k=1)

window_memory.save_context({"input": "Hello"}, {"output": "I'm fine, how are you?"})
window_memory.save_context({"input": "I want to ask about AI"}, {"output": "What do you want to know about AI?"})
window_memory.save_context({"input": "What is AI?"}, {"output": "AI is artificial intelligence."})

print("--- ConversationBufferWindowMemory Example (k=1) ---")
print(window_memory.buffer) # Only the last message remains

### 3.4. `ConversationSummaryMemory` (Summarizes History)

* **Concept:** Uses an LLM to periodically summarize the conversation history. Only the summary is stored instead of the raw messages.
* **Pros:** Retains long-term context without exceeding token limits, as only the summary is stored.
* **Cons:** Incurs additional cost and latency due to LLM calls for summarization. Summary quality depends on the LLM.
* **When to Use:** When you need to maintain long-term context and the conversation can be very long.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm_for_summary = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # LLM for summarization

# Initialize ConversationSummaryMemory
summary_memory = ConversationSummaryMemory(llm=llm_for_summary)

summary_memory.save_context({"input": "Hello"}, {"output": "I'm fine, how are you?"})
summary_memory.save_context({"input": "I want to ask about LangChain"}, {"output": "LangChain is a framework for building LLM applications."})
summary_memory.save_context({"input": "What are its components?"}, {"output": "Key components include Models, Prompts, Chains, Agents, Retrieval, Memory."})

print("--- ConversationSummaryMemory Example ---")
print(summary_memory.buffer) # View the summary stored

### 3.5. `ConversationSummaryBufferMemory` (Combines Window and Summary)

* **Concept:** Combines `ConversationBufferWindowMemory` and `ConversationSummaryMemory`. It stores the most recent messages in a window and summarizes older messages outside that window.
* **Pros:** Provides detailed context for recent messages and general context for the rest of the conversation, optimizing token usage.
* **When to Use:** When you want a balance between short-term detailed context and long-term general context.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm_for_summary_buffer = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize ConversationSummaryBufferMemory with max_token_limit
# It will summarize when the total token count exceeds the limit
summary_buffer_memory = ConversationSummaryBufferMemory(
    llm=llm_for_summary_buffer,
    max_token_limit=100 # Token limit for the buffer
)

summary_buffer_memory.save_context({"input": "Hello"}, {"output": "I'm fine, how are you?"})
summary_buffer_memory.save_context({"input": "I want to ask about LangChain"}, {"output": "LangChain is a framework for building LLM applications."})
summary_buffer_memory.save_context({"input": "What are its components?"}, {"output": "Key components include Models, Prompts, Chains, Agents, Retrieval, Memory."})
summary_buffer_memory.save_context({"input": "What is RAG?"}, {"output": "RAG is Retrieval-Augmented Generation."})
summary_buffer_memory.save_context({"input": "Why is RAG important?"}, {"output": "RAG helps reduce hallucinations and increase accuracy."})

print("--- ConversationSummaryBufferMemory Example ---")
print(summary_buffer_memory.buffer) # View the buffer (might have been summarized)


---

## 4. How to Integrate Memory into Chains and Agents

### 4.1. Integrating Memory into Chains

For Chains like `ConversationChain`, you simply pass the Memory object to the `memory` parameter when initializing the Chain. The Chain will automatically manage injecting the history into the prompt and updating the memory after each turn.

In [ ]:
# Example already illustrated in the ConversationBufferMemory section
# conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

### 4.2. Integrating Memory into Agents

For Agents, integrating Memory is slightly more complex because Agents need flexibility in using Tools. You need to add a `MessagesPlaceholder` to the Agent's `PromptTemplate` so the LLM can see the conversation history. Then, you pass the conversation history to the `AgentExecutor` via the `chat_history` parameter.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai google-search-results numexpr

import os
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.tools import Tool
from langchain_community.tools.calculator.tool import Calculator
from langchain.memory import ConversationBufferMemory # Use Memory for Agent

# Set environment variables for OpenAI and SerpAPI keys
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
# os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize Tools
search_tool = Tool(
    name="Google Search",
    func=SerpAPIWrapper().run,
    description="Useful when you need to search for information on Google about current events or factual data."
)
calculator_tool = Calculator()
tools = [search_tool, calculator_tool]

# Initialize Memory for Agent
agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define Prompt for Agent with MessagesPlaceholder for chat_history
prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You have access to the following tools: {tools}. Use them to answer questions. Maintain the conversation context."),
    MessagesPlaceholder(variable_name="chat_history"), # This is where the chat history will be injected
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create Agent
agent_with_memory = create_react_agent(llm, tools, prompt_with_memory)

# Create Agent Executor
agent_executor_with_memory = AgentExecutor(
    agent=agent_with_memory,
    tools=tools,
    verbose=True,
    memory=agent_memory # Pass the memory object to the Agent Executor
)

print("\n--- Agent with Memory Example ---")

# Conversation turn 1
query_1 = "What's the weather like today in Da Nang?"
print(f"You: {query_1}")
response_1 = agent_executor_with_memory.invoke({"input": query_1})
print(f"AI: {response_1['output']}")

# Conversation turn 2 (LLM should remember "Da Nang")
query_2 = "What's the average temperature there in July?"
print(f"You: {query_2}")
# Note: Here, the LLM might need to use the search tool again or infer from context.
# If the mock weather tool doesn't have enough info, it will say it doesn't know.
# For this example to work well, the web search tool (SerpAPI) would need to be used.
response_2 = agent_executor_with_memory.invoke({"input": query_2})
print(f"AI: {response_2['output']}")

print("\n--- History in Agent Memory ---")
print(agent_memory.load_memory_variables({}))


---

## Lesson Summary

This lesson addressed the inherent **statelessness problem** of LLMs and introduced the crucial concept of **Memory** in LangChain. You understood why Memory is essential for conversational applications and explored common Memory types such as `ChatMessageHistory`, `ConversationBufferMemory`, `ConversationBufferWindowMemory`, `ConversationSummaryMemory`, and `ConversationSummaryBufferMemory`, each with its own pros and cons for managing conversation history. Finally, you learned how to **integrate Memory into both Chains and Agents**, enabling your LLM applications to maintain context, resulting in a more natural and effective user experience.